In [2]:
!pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/c2/d7/90f34cb0d83a6c5631cf71dfe64cc1054598c843a92b400e55675cc2ac37/pip-18.1-py2.py3-none-any.whl
  Found existing installation: pip 18.0
    Uninstalling pip-18.0:
      Successfully uninstalled pip-18.0


In [3]:
!source activate tensroflow

/bin/sh: 1: source: not found


In [4]:
!pip install tensorflow

    100% |████████████████████████████████| 63.0MB 914kB/s 
  Using cached https://files.pythonhosted.org/packages/c2/f9/28787754923612ca9bfdffc588daa05580ed70698add063a5629d1a4209d/protobuf-3.6.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a7/9c/523fec4e50cd4de5effeade9fab6c1da32e7e1d72372e8e514274ffb6509/grpcio-1.15.0-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 3.0MB 2.3MB/s 
  Using cached https://files.pythonhosted.org/packages/35/6b/11530768cac581a12952a2aad00e1526b89d242d0b9f59534ef6e6a1752f/astor-0.7.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 51kB 1.5MB/s 
    100% |████████████████████████████████| 92kB 1.9MB/s 
    100% |████████████████████████████████| 92kB 238kB/s 
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/smajumdar/.cache/pip/wheels/3c/33/ae/db8cd618e62f87594c13a5483f96e618044f9b01596efd013f
Successfully built absl-py


In [5]:
import tensorflow as tf

/home/smajumdar/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
import math
import numpy as np

In [8]:
class Model(object) :
    def __init__(self,vocab_dim,emb_dim,margin=0.01) :
        self._vocab_dim = vocab_dim
        self._emb_dim = emb_dim
        self._random_seed = 42
        self._margin = margin
        
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)
        
        self.config = tf.ConfigProto()
        self.config.gpu_options.allow_growth = True
        self.build_model()
        
        self.saver = tf.train.Saver()
        
        
        init_op = tf.initialize_all_variables()
        with self.graph.as_default() :
            self.session.run(init_op)
        
    def build_model(self) :
        self.create_variables()
        tf.set_random_seed(self._random_seed + 1)
        
        A_var = tf.Variable(initial_value=tf.random_uniform(shape=[self._emb_dim,self._vocab_dim],minval=-1,maxval=1,seed=(self._random_seed + 2)))
        B_var = tf.Variable(initial_value=tf.random_uniform(shape=[self._emb_dim,self._vocab_dim],minval=-1,maxval=1,seed=(self._random_seed + 2)))
        
        self.global_step = tf.Variable(0,dtype=tf.int32,trainable=False,name='global_step')
        
        cont_mult = tf.transpose(tf.matmul(A_var,tf.transpose(self.context_batch)))
        resp_mult = tf.matmul(B_var,tf.transpose(self.response_batch))
        neg_resp_mult = tf.matmul(B_var,tf.transpose(self.neg_response_batch))
        
        pos_raw_f = tf.diag_part(tf.matmul(cont_mult,resp_mult))
        neg_raw_f = tf.diag_part(tf.matmul(cont_mult,neg_resp_mult))
        
        self.f_pos = pos_raw_f
        self.f_neg = neg_raw_f
        
        self.predict_op = tf.argmax(tf.nn.relu(self.f_neg - self.f_pos + self._margin))
        
        self.loss = tf.reduce_sum(tf.nn.relu(self.f_neg - self.f_pos + self._margin))
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(self.loss)
        
    def create_variables(self) :
        
        self.context_batch = tf.placeholder(dtype=tf.float32,name='Context',shape=[None,self._vocab_dim])
        self.response_batch = tf.placeholder(dtype=tf.float32,name='Response',shape=[None,self._vocab_dim])
        self.neg_response_batch = tf.placeholder(dtype=tf.float32,name='NegResponse',shape=[None,self._vocab_dim])
        
    def _init_summaries(self) :
        self.accuracy = tf.placeholder_with_default(0.0,shape=(),name='Accuracy')
        self.accuracy_summary = tf.scaler_summary('Accuracy summary',self.accuracy)
        
        self.f_pos_summary = tf.histogram_summary('f_pos',self.f_pos)
        self.f_neg_summary = tf.histogram_summary('f_neg',self.f_neg)
        
        self.loss_summary = tf.scaler_summary('Mini-batch loss',self.loss)
        
        self.summary_op = tf.merge_summary([self.f_pos_summary,self.f_neg_summary,self.loss_summary])
        
    def batch_fit(self,context_batch,response_batch,neg_response_batch) :
        feed_dict = {self.context_batch : context_batch, self.response_batch : response_batch, self.neg_response_batch : neg_response_batch}
        loss = self.session.run([self.loss,self.optimizer],feed_dict=feed_dict)
        return loss
    
    def batch_compute_loss(self,context_batch,response_batch,neg_response_batch) :
        feed_dict = {self.context_batch : context_batch, self.response_batch : response_batch, self.neg_response_batch : neg_response_batch}
        loss = self.session.run([self.loss],feed_dict=feed_dict)
        return loss
    